<a href="https://colab.research.google.com/github/Roni81/smartfarm/blob/main/growing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.3761647179999272
GPU (s):
0.07323069399990345
GPU speedup over CPU: 18x


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 24 21:57:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              41W / 300W |   1420MiB / 16384MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [9]:
import cv2
import os

def resize_images(input_folder, output_folder, target_size):
    # 출력 폴더가 없다면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 입력 폴더에서 이미지 파일 경로 리스트 가져오기
    image_paths = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    # 각 이미지에 대해 크기 조정 및 저장
    for image_path in image_paths:
        # 이미지 읽기
        image = cv2.imread(image_path)

        # 이미지가 비어 있지 않은지 확인
        if image is not None and not image.size == 0:
            # 이미지 크기 조정
            resized_image = cv2.resize(image, target_size)

            # 출력 폴더에 조정된 이미지 저장
            output_path = os.path.join(output_folder, os.path.basename(image_path))
            cv2.imwrite(output_path, resized_image)
        else:
            print(f"이미지를 읽거나 크기를 조정할 수 없습니다: {image_path}")

# 예시: 입력 폴더, 출력 폴더, 타겟 크기 설정
input_folder = "/content/drive/MyDrive/growing2_temp/growing2_temp/images"
output_folder = "/content/drive/MyDrive/growing2_temp/growing2_temp/images_resized"
target_size = (1000, 750)  # 원하는 크기로 변경

# 이미지 크기 조정 실행
resize_images(input_folder, output_folder, target_size)

In [9]:
# 이미지 데이터 로드 및 전처리
def load_images_and_metadata(image_paths, metadata_folder):
    images = []
    metadata_list = []

    for img_path in image_paths:
        # 이미지 읽기
        img = cv2.imread(img_path)

        # 이미지가 성공적으로 로드되었는지 확인
        if img is not None:
            # 이미지 정규화 및 리스트에 추가
            img = img / 255.0
            images.append(img)

            # 이미지 파일 이름에서 식별자 추출 (예: 이미지 파일이름이 '123.jpg'이면 식별자는 '123')
            identifier = os.path.splitext(os.path.basename(img_path))[0]

            # 식별자를 사용하여 메타데이터 파일 찾기 및 읽기
            metadata_file_path = os.path.join(metadata_folder, f"{identifier}.csv")
            metadata = pd.read_csv(metadata_file_path)
            metadata_list.append(metadata)
        else:
            print(f"경고: {img_path}에서 이미지를 읽을 수 없습니다.")

    # 이미지 데이터와 메타데이터를 연결
    images = np.array(images)
    metadata = pd.concat(metadata_list, ignore_index=True)

    return images, metadata
# 메타데이터 및 라벨 데이터 로드
def load_labels(labels_path):
    labels = pd.read_csv(labels_path)
    return labels

In [10]:
# 데이터셋 경로 설정
image_paths = glob("/content/drive/MyDrive/growing2_temp/growing2_temp/images_resized/*.jpg")
labels_path = "/content/drive/MyDrive/growing2_temp/growing2_temp/combined_dataset.csv"
metadata_folder = "/content/drive/MyDrive/growing2_temp/growing2_temp/metas/"



In [ ]:
# 이미지, 메타데이터, 라벨 로드
images, metas = load_images_and_metadata(image_paths, metadata_folder)
labels = load_metadata_and_labels(labels_path)


CNN RNN 앙상블

In [ ]:
# 이미지 CNN 모델
image_input = Input(shape=(height, width, channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
maxpool1 = MaxPooling2D((2, 2))(conv1)
flatten1 = Flatten()(maxpool1)

# 메타데이터 RNN 모델
meta_input = Input(shape=(num_metadata_features,))
embedding = Embedding(input_dim=num_categories, output_dim=embedding_dim)(meta_input)
lstm1 = LSTM(64)(embedding)

# 모델 병합
merged = concatenate([flatten1, lstm1])
dense1 = Dense(128, activation='relu')(merged)
output = Dense(num_classes, activation='softmax')(dense1)

model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit([X_image_train, X_meta_train],
          to_categorical(y_train),
          epochs=10, batch_size=32,
          validation_data=([X_image_val, X_meta_val], to_categorical(y_val)))

In [ ]:
predictions = model.predict([X_image_val, X_meta_val])

CNN_LSTM

In [ ]:
# 이미지 CNN 모델
image_input = Input(shape=(height, width, channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
maxpool1 = MaxPooling2D((2, 2))(conv1)
flatten1 = Flatten()(maxpool1)

# 메타데이터 LSTM 모델
meta_input = Input(shape=(num_metadata_features,))
embedding = Embedding(input_dim=num_categories, output_dim=embedding_dim)(meta_input)
lstm1 = LSTM(64)(embedding)

# 모델 병합
merged = concatenate([flatten1, lstm1])
dense1 = Dense(128, activation='relu')(merged)
output = Dense(num_classes, activation='softmax')(dense1)

model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit([X_image_train, X_meta_train],
          to_categorical(y_train),
          epochs=10, batch_size=32,
          validation_data=([X_image_val, X_meta_val], to_categorical(y_val)))

Multi_Modal Neural Network